In [211]:
import altair as alt
import pandas as pd
from IPython.display import display

csv_path = "C:/Users/bruno/Code/pixelweave/build/benchmark/benchmark.csv"
csv_data = pd.read_csv(csv_path)
display(csv_data.head(5))

,InputFormat,InputWidth,InputHeight,OutputFormat,OutputWidth,OutputHeight,CopyToDeviceVisibleTimeMicros,TransferDeviceVisibleToDeviceLocalTimeMicros,ComputeConversionTimeMicros,CopyDeviceVisibleToHostLocalTimeMicros,TotalTime
0,Interleaved8BitUYVY,3840,2160,Planar8Bit420,3840,2160,3043,1779,617,45287,50726
1,Interleaved8BitUYVY,3840,2160,Planar8Bit422,3840,2160,3081,1845,702,54620,60248
2,Interleaved8BitUYVY,3840,2160,Planar8Bit444,3840,2160,1964,1506,667,65346,69483
3,Interleaved8BitUYVY,3840,2160,Planar10Bit420,3840,2160,1964,1578,657,65185,69384
4,Interleaved8BitUYVY,3840,2160,Planar10Bit422,3840,2160,1891,1524,552,87113,91080


In [218]:
df = csv_data[(csv_data["InputWidth"] == csv_data["OutputWidth"])]

input_formats = df["InputFormat"].unique()
input_formats_dropdown = alt.binding_select(options=input_formats, name="Input format")
input_formats_select = alt.selection_single(
    fields=["InputFormat"],
    bind=input_formats_dropdown,
    name="Input format",
    init={'InputFormat': input_formats[1]}
)

output_formats = df["OutputFormat"].unique()
output_formats_dropdown = alt.binding_select(options=output_formats, name="Output format")
output_formats_select = alt.selection_single(
    fields=["OutputFormat"],
    bind=output_formats_dropdown,
    name="Output format",
    init={'OutputFormat': output_formats[1]}
)

columns = [
    "CopyToDeviceVisibleTimeMicros:Q",
    "TransferDeviceVisibleToDeviceLocalTimeMicros:Q",
    "ComputeConversionTimeMicros:Q",
    "CopyDeviceVisibleToHostLocalTimeMicros:Q",
]

colors = [
    'navy', 'chocolate', 'darkseagreen', 'fuchsia'
]

base = alt.Chart(df).mark_area(size=1).encode(
    x=alt.X("OutputWidth:Q", title="Width"),
    y=alt.Y("TotalTime:Q", title="Time (Micros)"),
    opacity=alt.value(0.3)
)

layers = [
    alt.Chart(df).mark_line().encode(
        x=alt.X("OutputWidth:Q"),
        y=alt.Y(column),
        tooltip=[column],
        color=alt.value(color)
    )
    for column, color in zip(columns, colors)
]

alt.layer(
    *[base, *layers]
).resolve_scale(
    y="shared",
    color="independent"
).add_selection(
    input_formats_select, output_formats_select
).transform_filter(
    input_formats_select & output_formats_select
).interactive()

alt.LayerChart(...)